In [ ]:
import numpy as np #импортирование нужных для работы библиотек
import pandas


In [ ]:
data = pandas.read_csv('minutes_n_ingredients.csv') #1 задание
data

,id,minutes,n_ingredients
0,127244,60,16
1,23891,25,7
2,94746,10,6
3,67660,5,6
4,162911,60,14
...,...,...,...
99995,43407,35,7
99996,287943,85,15
99997,498432,65,15
99998,380915,5,4


Файл minutes_n_ingredients.csv содержит информацию об идентификаторе рецепта, времени его выполнения в минутах и количестве необходимых ингредиентов. Считайте данные из этого файла в виде массива numpy типа int32, используя np.load_txt. Выведите на экран первые 5 строк массива.


In [ ]:
data = np.loadtxt('minutes_n_ingredients.csv', delimiter=',', skiprows=1, dtype=np.int32)
data[:5]

array([[127244,     60,     16],
       [ 23891,     25,      7],
       [ 94746,     10,      6],
       [ 67660,      5,      6],
       [162911,     60,     14]])

Вычислите среднее значение, минимум, максимум и медиану по каждому из столбцов, кроме первого.

In [ ]:
print(data[:, 1:].mean(axis=0), data[:, 1:].min(axis=0), data[:, 1:].max(axis=0), np.median(data[:, 1:], axis=0), sep='\n')


[2.1601398e+04 9.0552800e+00]
[0 1]
[2147483647         39]
[40.  9.]


Ограничьте сверху значения продолжительности выполнения рецепта значением квантиля  q0.75 .


In [ ]:
q = np.quantile(data[:, 1], q=0.75)
data[:, 1] = data[:, 1].clip(max=q)
q

65.0

Посчитайте, для скольких рецептов указана продолжительность, равная нулю. Замените для таких строк значение в данном столбце на 1.

In [ ]:
print(data[data[:, 1]==0].shape[0])
data[data[:, 1]==0, 1]=1

479


Посчитайте, сколько уникальных рецептов находится в датасете.


In [ ]:
len(np.unique(data[:, 0], axis=0))

98252

Сколько и каких различных значений кол-ва ингредиентов присутвует в рецептах из датасета?

In [ ]:
len(np.unique(data[:, 2]))

37

In [ ]:
np.unique(data[:, 2])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 37, 39])

Создайте версию массива, содержащую информацию только о рецептах, состоящих не более чем из 5 ингредиентов.

In [ ]:
data_n_ingr_m5 = data[data[:, 2] <= 5].copy()
data_n_ingr_m5

array([[446597,     15,      5],
       [204134,      5,      3],
       [ 25623,      6,      4],
       ...,
       [ 52088,     60,      5],
       [128811,     15,      4],
       [380915,      5,      4]])

Для каждого рецепта посчитайте, сколько в среднем ингредиентов приходится на одну минуту рецепта. Найдите максимальное значение этой величины для всего датасета

In [ ]:
medium_ingred_by_minutes = data[:, 2] / data[:, 1]
medium_ingred_by_minutes.mean(), medium_ingred_by_minutes.max()

(0.39829031572089063, 24.0)

Вычислите среднее количество ингредиентов для топ-100 рецептов с наибольшей продолжительностью

In [ ]:
data[data[:, 1].argsort()][-100:][:, 2].mean()

9.75

Выберите случайным образом и выведите информацию о 10 различных рецептах

In [ ]:
random_ind = np.random.randint(0, 10000, size=10)
data[random_ind]

array([[290216,     25,      8],
       [514895,     30,      5],
       [373158,      5,      6],
       [114103,     65,     11],
       [380078,     50,     11],
       [464901,     38,      8],
       [ 96276,     35,     11],
       [388617,     65,     13],
       [236938,     25,      6],
       [ 40843,     45,      7]])

Выведите процент рецептов, кол-во ингредиентов в которых меньше среднего.

In [ ]:
sred_ingred = data[:, 2].mean()
len(data[data[:, 2] < sred_ingred]) / len(data) * 100

58.802

Назовем "простым" такой рецепт, длительность выполнения которого не больше 20 минут и кол-во ингредиентов в котором не больше 5. Создайте версию датасета с дополнительным столбцом, значениями которого являются 1, если рецепт простой, и 0 в противном случае.


In [ ]:
prostota = (data[:, 1] <= 20) & (data[:, 2] <= 5)
prostota = prostota.astype(np.int32)
prostota = prostota[:, np.newaxis]

data_prostota = np.hstack((data, prostota))
data_prostota

array([[127244,     60,     16,      0],
       [ 23891,     25,      7,      0],
       [ 94746,     10,      6,      0],
       ...,
       [498432,     65,     15,      0],
       [380915,      5,      4,      1],
       [ 81993,     65,     14,      0]])

Выведите процент "простых" рецептов в датасете

In [ ]:
len(data_prostota[data_prostota[:, 3] == 1]) / len(data_prostota) * 100

9.552

Разделим рецепты на группы по следующему правилу. Назовем рецепты короткими, если их продолжительность составляет менее 10 минут; стандартными, если их продолжительность составляет более 10, но менее 20 минут; и длинными, если их продолжительность составляет не менее 20 минут. Создайте трехмерный массив, где нулевая ось отвечает за номер группы (короткий, стандартный или длинный рецепт), первая ось - за сам рецепт и вторая ось - за характеристики рецепта. Выберите максимальное количество рецептов из каждой группы таким образом, чтобы было возможно сформировать трехмерный массив. Выведите форму полученного массива.

In [ ]:
short = data[data[:, 1] < 10]
standart = data[(data[:, 1] >= 10) & (data[:, 1] < 20)]
long = data[data[:, 1] >= 20]
stop = min([len(arr) for arr in [short, standart, long]])
np.array([short[:stop], standart[:stop], long[:stop]])

array([[[ 67660,      5,      6],
        [366174,      7,      9],
        [204134,      5,      3],
        ...,
        [420725,      5,      3],
        [  4747,      1,      9],
        [380915,      5,      4]],

       [[ 94746,     10,      6],
        [ 33941,     18,      9],
        [446597,     15,      5],
        ...,
        [  9831,     15,      7],
        [335859,     12,     14],
        [256812,     10,      3]],

       [[127244,     60,     16],
        [ 23891,     25,      7],
        [162911,     60,     14],
        ...,
        [168901,     25,      7],
        [392339,     35,     13],
        [206732,     45,     10]]])